# Sistemas Inteligentes práctica 1
## Generación de texto predictivo basado en N-grams
### 2020-2021
    

In [1]:
# Práctica basada en:
# https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/
# https://nlpforhackers.io/language-models/
    
# Más información en:
# https://www.nltk.org/api/nltk.html
# https://web.stanford.edu/~jurafsky/slp3/ (--> Language Modeling with N-Grams)

# Libros a descargar en texto plano UTF-8:
# https://www.gutenberg.org/

In [2]:
from nltk import bigrams, trigrams, ngrams
from collections import defaultdict

## Trigrams model

In [3]:
# Create a placeholder for model
modelTrigrams = defaultdict(lambda: defaultdict(lambda: 0))

log=1

#Get the text

with open("pg2000.txt", "r", encoding="utf-8") as myfile:
    texto = myfile.read()

texto='hola esto es un ejemplo que lo escribo yo mismo. hola esto puede ser otro ejemplo o es un coche o es rojo. hola esto debe ser un ejemplo. Aunque no lo tengo tan claro, sabes? Qué te parece el resultado?'    

#Sist Intel práctica 1: Introducimos el texto primero en un string con el fin de comprender cómo funcionan,
#para posteriormente poder trabajar con ficheros de texto

#Prepare the text
texto=texto[:] #consider to delete the head and what you consider, goal is to get the pure text from the "raw" text
texto=texto.replace("."," . ")
texto=texto.replace(","," , ")
texto=texto.replace("?"," ? ")
texto=texto.replace("¿"," ¿ ")
texto=texto.replace("!"," ! ")
texto=texto.replace("¡"," ¡ ")
#Sist Intel práctica 1: ¿Por qué es necesaria la preparación del texto?

#Get the text as list of words
texto2=texto.split()

# Build the model based on Trigrams
for w1, w2, w3 in trigrams(texto2, pad_right=True, pad_left=True):
        if log:
            print(w1, w2, w3)
        modelTrigrams[(w1, w2)][w3] += 1
        
# Let's transform the counts to probabilities
for w1_w2 in modelTrigrams:
    total_count = float(sum(modelTrigrams[w1_w2].values()))
    for w3 in modelTrigrams[w1_w2]:
        modelTrigrams[w1_w2][w3] /= total_count     
        
#Sist Intel práctica 1: ¿Puedes explicar qué y con qué finalidad se hace el código de 
# la construcción del modelo y en su transformación a probabilidades? ¿Por qué aparecen None en los trigrams? ten
#en cuenta los parámetros: trigrams(texto2, pad_right=True, pad_left=True)

None None hola
None hola esto
hola esto es
esto es un
es un ejemplo
un ejemplo que
ejemplo que lo
que lo escribo
lo escribo yo
escribo yo mismo
yo mismo .
mismo . hola
. hola esto
hola esto puede
esto puede ser
puede ser otro
ser otro ejemplo
otro ejemplo o
ejemplo o es
o es un
es un coche
un coche o
coche o es
o es rojo
es rojo .
rojo . hola
. hola esto
hola esto debe
esto debe ser
debe ser un
ser un ejemplo
un ejemplo .
ejemplo . Aunque
. Aunque no
Aunque no lo
no lo tengo
lo tengo tan
tengo tan claro
tan claro ,
claro , sabes
, sabes ?
sabes ? Qué
? Qué te
Qué te parece
te parece el
parece el resultado
el resultado ?
resultado ? None
? None None


In [4]:
#predict the next word
dict(modelTrigrams["es", "un"])

#Sist Intel práctica 1: Realiza diferentes pruebas con secuencias de dos palabras que se encuentren seguidas, ¿tiene sentido?

{'ejemplo': 0.5, 'coche': 0.5}

In [5]:
# List trigrams with calculated probabilities
numIter = 0
numIterMax = 20
for x1,x2 in modelTrigrams:
    print(x1,x2, ': ', dict(modelTrigrams[x1, x2]))
    numIter+=1
    if numIter >= numIterMax:
        break
        
 #Sist Intel práctica 1: ¿Qué significa el resultado obtenido? ¿Por qué?

None None :  {'hola': 1.0}
None hola :  {'esto': 1.0}
hola esto :  {'es': 0.3333333333333333, 'puede': 0.3333333333333333, 'debe': 0.3333333333333333}
esto es :  {'un': 1.0}
es un :  {'ejemplo': 0.5, 'coche': 0.5}
un ejemplo :  {'que': 0.5, '.': 0.5}
ejemplo que :  {'lo': 1.0}
que lo :  {'escribo': 1.0}
lo escribo :  {'yo': 1.0}
escribo yo :  {'mismo': 1.0}
yo mismo :  {'.': 1.0}
mismo . :  {'hola': 1.0}
. hola :  {'esto': 1.0}
esto puede :  {'ser': 1.0}
puede ser :  {'otro': 1.0}
ser otro :  {'ejemplo': 1.0}
otro ejemplo :  {'o': 1.0}
ejemplo o :  {'es': 1.0}
o es :  {'un': 0.5, 'rojo': 0.5}
un coche :  {'o': 1.0}


In [6]:
import random

# starting words
palabra1 = 'hola'
palabra2 = 'esto'
text = [palabra1, palabra2]

errorNoAvailable = 0

if dict(modelTrigrams[text[0], text[1]])=={}:
    print('error: secuencia no encontrada')
    errorNoAvailable = 1

sentence_finished = False
numberOfwords = 0
maxNumberofWords = 10
 
while (not sentence_finished) and errorNoAvailable==0:
    # select a random probability threshold  
    r = random.random()
    #log:
    if log:
        print('umbral random: ', r)
    accumulator = .0
    
    for word in modelTrigrams[tuple(text[-2:])].keys():
        accumulator += modelTrigrams[tuple(text[-2:])][word]
        #log:
        if log:
            print('palabras anteriores -->', tuple(text[-2:]))
            print('palabra: ', word, ', probabilidad de palabra: ', modelTrigrams[tuple(text[-2:])][word],' --> probabilidad acumulada: ', accumulator)        
                      
        # select words that are above the probability threshold
        if accumulator >= r:
            numberOfwords+=1
            text.append(word)
            #log:
            if log:
                print('\tpalabra buena: ==>', word)
                print('frase construida: ', ' '.join([t for t in text if t]))
            break
    if text[-2:] == [None, None]:
        sentence_finished = True
        
    if numberOfwords>=maxNumberofWords:
        sentence_finished = True
 
if errorNoAvailable==0:
    print (' '.join([t for t in text if t]))

#Sist Intel práctica 1: Explicar cómo funciona la generación de frases, desde la variable r, accumulator, puedes utilizar el log obtenido
# para interepretar el algoritmo

umbral random:  0.9227224338738217
palabras anteriores --> ('hola', 'esto')
palabra:  es , probabilidad de palabra:  0.3333333333333333  --> probabilidad acumulada:  0.3333333333333333
palabras anteriores --> ('hola', 'esto')
palabra:  puede , probabilidad de palabra:  0.3333333333333333  --> probabilidad acumulada:  0.6666666666666666
palabras anteriores --> ('hola', 'esto')
palabra:  debe , probabilidad de palabra:  0.3333333333333333  --> probabilidad acumulada:  1.0
	palabra buena: ==> debe
frase construida:  hola esto debe
umbral random:  0.23433140397375463
palabras anteriores --> ('esto', 'debe')
palabra:  ser , probabilidad de palabra:  1.0  --> probabilidad acumulada:  1.0
	palabra buena: ==> ser
frase construida:  hola esto debe ser
umbral random:  0.3422969218498324
palabras anteriores --> ('debe', 'ser')
palabra:  un , probabilidad de palabra:  1.0  --> probabilidad acumulada:  1.0
	palabra buena: ==> un
frase construida:  hola esto debe ser un
umbral random:  0.84597375289

## Bigrams model

In [7]:
#Sist Intel práctica 1: Extiende el model de trigrams a bigrams, puedes copiar las mismas celdas
# del modelo basado en trigrams y adaptarlo

In [8]:
#Sist Intel práctica 1: Puedes comparar y extraer conclusiones de las frases generadas en trigrams y bigrams?
#Sist Intel práctica 1: Puedes utilizar diferentes fuentes de datos (libros) y extraer conclusiones
# de las frases generadas en trigrams y bigrams en función de as fuentes?

# Ngrams model

In [ ]:
%pip install gutenberg-cleaner
from gutenberg_cleaner import simple_cleaner, super_cleaner

In [11]:
from sklearn.model_selection import train_test_split
import re
import multiprocessing
from itertools import chain
from operator import methodcaller
import numpy as np
import random
from nltk import ngrams
from collections import defaultdict
import pydot
from matplotlib import pyplot as plt


In [12]:
def draw_graph(model, filename):
    if len(model) > 50:
        print("Model is too big")
    else:
        # Draw the graph
        graphnodes = []
        written = []
        def traverse(w):
            for nw in model[w]:
                if (w[-1],nw) not in written:
                    graphnodes.append(f"\"{w[-1]}\" [label=\"{w[-1]}\"];")
                    graphnodes.append(f"\"{nw}\" [label=\"{nw}\"];")
                    path_label = f"({w[0:-1]})" if len(w[0:-1]) else ""
                    graphnodes.append(f"\"{w[-1]}\" -> \"{nw}\" [label=\"{path_label}{model[w][nw]:.2f}\"];")
                    written.append((w[-1],nw))
                    traverse((w[1:])+(nw,))

        for w in model:
            traverse(w)

        dot_string = "digraph G{\nrankdir=LR\n" + "\n".join(graphnodes) + "}"
        graph = pydot.graph_from_dot_data(dot_string)
        with open(f"./{filename}.pdf", 'wb') as f:
            f.write(graph[0].create_pdf())
        graph[0].write_png(f"./{filename}.png")
        graph_png = cv2.imread(f"./{filename}.png",1)
        height, width, channels = graph_png.shape
        plt.figure(figsize = (width/96,height/96),dpi=96)
        plt.imshow(graph_png)
        plt.axis("off")
        plt.show()

In [13]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ln -s /content/drive/MyDrive/sistint_p1 data

In [14]:
#modify according to your base path
# base_path="./data/"

In [15]:
# Split sentences by dot
def split_sentences(text):
  # From: https://stackoverflow.com/a/25736082
  # https://regex101.com/r/nG1gU7/27
  return re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',text)

# Clean input text, spaces between symbols
def text_cleaner(text):
  clean_text = text
  clean_text = clean_text.replace("\'","")
  clean_text = clean_text.replace("\"","")
  clean_text = clean_text.replace("—","")
  clean_text = clean_text.replace("♫","")
  clean_text = clean_text.replace("#","")
  clean_text = clean_text.replace("♪","")
  clean_text = clean_text.replace("«","")
  clean_text = clean_text.replace("»","")
  clean_text = re.sub(r"\([^\)]+\)",'',clean_text)
  clean_text = re.sub(r"([¿¡!\$%&\*\+,-\./:;<=>\?@\\\^_`{\|}~])",r" \1 ",clean_text)
  return clean_text

In [16]:

# Train ngram model
def train_ngram_model(words, n=3):
  model = defaultdict(lambda: defaultdict(lambda: 0))
  # Build the model based on ngrams
  for w in ngrams(words, n, pad_right=True, pad_left=True):
    model[w[0:-1]][w[-1]] += 1

  # Let's transform the counts to probabilities
  for w in model:
    total_count = float(sum(model[w].values()))
    for nw in model[w]:
      model[w][nw] /= total_count  
  return model

# Generate sentences (dot ended) with ngram model
def generate_sentences(begins_with, ngram_model, number_of_sentences=3):
    
  errorNoAvailable = 0
  
  # Obtain ngram grade
  ngram_model_n = len(list(ngram_model.keys())[0])

  text = begins_with

  # Look for a match
  if dict(ngram_model[tuple(text[-(ngram_model_n):])])=={}:
      print('error: secuencia no encontrada')
      return []

  # Sentence counter
  sentences_generated = 0

  # Loop til number of sentences
  while sentences_generated < number_of_sentences:
    # Generate a random number
    r = random.random()

    # Accumulated probability
    accumulator = .0
  
    for word in ngram_model[tuple(text[-(ngram_model_n):])].keys():
      accumulator += ngram_model[tuple(text[-(ngram_model_n):])][word]  
      # Select words that are above the random probability threshold
      # Uniform probability of select one of the paths
      if accumulator >= r:
          text.append(word)
          break
    if word in (None, '.') :      
      sentences_generated = sentences_generated + 1
  return " ".join([word for word in text if word])

# Calculate perplexity of a given sentence for a given model
def perplexity(sentence, ngram_model):
  ngram_model_n = len(list(ngram_model.keys())[0])
  P = list()
  N = len(sentence)
  for i in range(ngram_model_n,len(sentence)):
    if tuple(sentence[i-ngram_model_n:i]) in ngram_model and \
       sentence[i] in ngram_model[tuple(sentence[i-ngram_model_n:i])]:
      P.append(ngram_model[tuple(sentence[i-ngram_model_n:i])][sentence[i]])
    else:
      return "Cannot found this path"
  PP = np.power(np.reciprocal(np.prod(P)),1/N)
  return PP

In [17]:
# Don Quijote
# with open(base_path+"pg2000.txt", "r", encoding="utf-8") as myfile:
#     text = super_cleaner(myfile.read())
# text = text.replace("\n", " ")
# text = text.replace("[deleted]", "")
# sentences = split_sentences(text)[1:]

In [21]:
# From https://www.kaggle.com/datasets/miguelcorraljr/ted-ultimate-dataset
! wget -q --no-check-certificate -O 'ted_talks_es.csv' 'https://docs.google.com/uc?export=download&id=1C8QDQ9ecuSHvYeRQtB5LVfhBrfYIoBiZ'

In [22]:
## Ted Talks
import csv
transcripts = []
with open('ted_talks_es.csv') as csvfile:
    ted_reader = csv.DictReader(csvfile)
    sentences = [split_sentences(row['transcript']) for row in ted_reader]
    sentences = list(chain(*sentences))

In [23]:
# text='hola esto es un ejemplo que lo escribo yo mismo. hola esto puede ser otro ejemplo o es un coche o es rojo. hola esto debe ser un ejemplo. Aunque no lo tengo tan claro, sabes? Qué te parece el resultado?'    
# sentences = split_sentences(text)

In [24]:
# text = "Hugo tuvo un tubo, pero el tubo que tuvo se le rompió. Para recuperar el tubo que tuvo, tuvo que comprar un tubo igual al tubo que tuvo."
# sentences = split_sentences(text)

In [25]:
# Split train test sentences and clean them
s_train, s_test = train_test_split(sentences, test_size=0.05)
print(f"s_train: {len(s_train)} s_test: {len(s_test)}")
pool_obj = multiprocessing.Pool()
test_sentences = pool_obj.map(text_cleaner,s_test)
train_sentences = pool_obj.map(text_cleaner,s_train)

# Join sentences as validation doesn't work properly
train_sentences += test_sentences

s_train: 416746 s_test: 21935


In [26]:
# train_sentences = train_sentences + test_sentences
# Generate training words and flat the array
train_words = list(map(methodcaller("split", None), train_sentences))
train_words = list(chain(*train_words))

In [27]:
# Generate models
qgram_model = train_ngram_model(train_words, n =4)
trigram_model = train_ngram_model(train_words,n=3)
bigram_model = train_ngram_model(train_words,n=2)

In [28]:
draw_graph(bigram_model, "bigram")
draw_graph(trigram_model, "trigram")
draw_graph(qgram_model, "qgram")

Model is too big
Model is too big
Model is too big


In [29]:
print(generate_sentences("tuvo un tubo".split(), bigram_model, 3))
print(generate_sentences("tuvo un tubo".split(), trigram_model, 3))
print(generate_sentences("tuvo un tubo".split(), qgram_model, 3))

tuvo un tubo comienza a su madre de neuronas en el hecho que observar en búsqueda indexada es una manera que la arquitectura es Özlem . Ahora les gusta . Y es Satanás .
tuvo un tubo . Quizá deberíamos utilizar una sola nación . Y han llegado del todo seguro donde los fabricantes de base y se endurecen de forma estructurada y probar , procesar y exportaron productos de desecho que se censure .
error: secuencia no encontrada
[]


In [30]:
print(generate_sentences("esto es un ejemplo".split(), bigram_model, 3))
print(generate_sentences("esto es un ejemplo".split(), trigram_model, 3))
print(generate_sentences("esto es un ejemplo".split(), qgram_model, 3))

esto es un ejemplo , en direcciones opuestas que piensan en varias personas creaban herramientas es el menú del tiempo . Es la adversidad no buscan el que ocultar secretos fácilmente son los elefantes , y sin nunca se escape . Usamos energía , copias de las ruedas , días y asiático , retengan nuestros recursos , beautiful rechaza el caminar .
esto es un ejemplo . La obra consta de dos mundos tridimensionales realmente se diferencian terminalmente . Basándonos en miles de personas cuyo trabajo es extraordinario no es la que enseñen esto a través de sus nuevos vehiculos serían flex - fuel compatibles , y estaba siendo aplastada , y en aumento en la arqueología nos brinda una gran decepción de esperanzas , de Star Wars .
esto es un ejemplo característico de la humanidad , al menos , a mí sí . Sabemos que el cambio tecnológico en sí . Y dice , No .


In [31]:
print(generate_sentences("me gustaría mucho".split(), bigram_model, 2))
print(generate_sentences("me gustaría mucho".split(), trigram_model, 2))
print(generate_sentences("me gustaría mucho".split(), qgram_model, 2))

me gustaría mucho más lo que yo me fue simplemente extendió como un mundo un balón de salud en mi rumbo . Y un videoblog con suerte se moverá un bizcocho de tecnología para esto por ahí para interactuar con estas 1 , a la pérdida de la vivienda , o deslegitimar las elecciones durante dos metros , quiere decir , supongamos que define como la incipiente .
me gustaría mucho escuchar sus cuentos y también para nuestra economía , realmente sólo la mitad del escenario . Sus teléfonos se está llevando a cabo por cualquier acto de conmemoración camuflada en el norte de nuesto personal fué sigificativamente mayo que habría sido tratado con antibiótico es simplemente recordar que cada lado .
me gustaría mucho que comencemos a tumbarlos y a olvidar cosas . Pero , ¿ quién ? El conocimiento no tiene que ocurrir .


In [32]:
print(generate_sentences("Porque somos parte".split(), bigram_model, 2))
print(generate_sentences("Porque somos parte".split(), trigram_model, 2))
print(generate_sentences("Porque somos parte".split(), qgram_model, 2))

Porque somos parte posterior , especialmente cuando no es analizar una extremidad para llegar a capturarlo allí hace que tendré gente en encontrar jóvenes con Ud . Amigo , en general : Sí .
Porque somos parte de esto es un gran rendimiento . La gente pobre no puede ser demostrado , contribuyen al bien público .
Porque somos parte de la historia . Gracias .


In [33]:
ts = "El propósito y la alegría de la ciencia y la ingeniería ."
print(ts)
print(perplexity(ts.split(), qgram_model))
print(perplexity(ts.split(), trigram_model))
print(perplexity(ts.split(), bigram_model))

El propósito y la alegría de la ciencia y la ingeniería .
2.5685399630859127
13.645760650251104
44.702804712732366


In [34]:
ts = "Y eso es importante no solo de su verdadera vida psicológica"
print(perplexity(ts.split(), qgram_model))
print(perplexity(ts.split(), trigram_model))
print(perplexity(ts.split(), bigram_model))

3.898817599538185
14.42645186106499
101.67247517650911


In [35]:
ts = "Mira , tengo estos nuevos submarinos nucleares que estoy construyendo"
print(perplexity(ts.split(), qgram_model))
print(perplexity(ts.split(), trigram_model))
print(perplexity(ts.split(), bigram_model))

1.335141362540313
8.862856327032796
76.05991219113524
